# California Water Conservation Data Cleaning 
---

[Public Water System Operations Monthly Water Production and Conservation Information](https://data.ca.gov/dataset/drinking-water-public-water-system-operations-monthly-water-production-and-conservation-information)

[Data Dictionary](https://data.ca.gov/dataset/drinking-water-public-water-system-operations-monthly-water-production-and-conservation-information/resource/a3aa420d-3553-4891-8bc4-ed01aeaf3e20)

---
[Water Conservation Supplier Compliance](https://www.kaggle.com/calepa/water-conservation-supplier-compliance?select=supplier_compliance.csv)

#### Imports

In [776]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import datetime
import time
import matplotlib.pyplot as plt
import seaborn as sns

# import psutil
from multiprocess import Pool
from tqdm import tqdm
sns.set(style="ticks", context="talk")
plt.style.use("dark_background")

In [777]:
# df = pd.read_csv('./datasets/9_GFW_potential_carbon_sequestration/young_forest_sequestration_rate_Griscom_extent.tif')
conserv = pd.read_csv('./datasets/conservation_compliance/supplier_compliance.csv')
july = pd.read_csv('./datasets/conservation_compliance/july_2016_nonfilers.csv')
august = pd.read_csv('./datasets/conservation_compliance/august_2016_nonfilers.csv')

water = pd.read_csv('./datasets/water_capacity.csv')

---
## CA Water Conservation Compliance Data
---

In [778]:
conserv.head()

,Supplier Name,Hydrologic Region,State-mandated Conservation Standard - Previous (3/1/2016),State-mandated Conservation Standard - Current (Effective 6/1/2016),Did Supplier Self-certify?,Enforceable Cumulative Percent Savings (as compared to 2013) - June 2015-Aug 2016,June 2016 to Aug 2016 Self-certified?,Applicable Cumulative Savings by Supplier (as compared to 2013),Missed Conservation Standard By,Order Status,Monthly Savings (as compared to 2013) - Aug 2015,Monthly Savings (as compared to 2013) - Aug 2016,Estimated Aug 2016 Monthly R-GPCD,Total Population Served
0,California City City of,South Lahontan,28%,28%,No,13.1%,18.0%,13.1%,14.9%,C,16.8%,20.5%,148.5,"14,120"
1,Yucaipa Valley Water District,South Coast,34%,20%,Yes,18.3%,7.1%,7.1%,12.9%,NaN,26.4%,11.9%,260.0,"45,018"
2,Sanger City of,Tulare Lake,26%,26%,No,17.3%,12.4%,17.3%,8.7%,NaN,38.0%,17.2%,181.1,"25,664"
3,Adelanto City of,South Lahontan,16%,16%,No,9.3%,-10.3%,9.3%,6.7%,C,20.5%,-30.6%,125.5,"29,023"
4,Red Bluff City of,Sacramento River,33%,33%,No,27.8%,19.5%,27.8%,5.2%,NaN,42.4%,30.8%,202.9,"14,076"


In [779]:
conserv.columns

Index(['Supplier Name', ' Hydrologic Region',
       ' State-mandated Conservation Standard - Previous (3/1/2016)',
       ' State-mandated Conservation Standard - Current (Effective 6/1/2016)',
       ' Did Supplier Self-certify?',
       ' Enforceable Cumulative Percent Savings (as compared to 2013) - June 2015-Aug 2016',
       ' June 2016 to Aug 2016 Self-certified?',
       ' Applicable Cumulative Savings by Supplier (as compared to 2013)',
       ' Missed Conservation Standard By', ' Order Status',
       ' Monthly Savings (as compared to 2013) - Aug 2015',
       ' Monthly Savings (as compared to 2013) - Aug 2016',
       ' Estimated Aug 2016 Monthly R-GPCD', ' Total Population Served'],
      dtype='object')

In [780]:
print("conserv shape", conserv.shape)
print("jul shape", july.shape)
print("aug shape", august.shape)

conserv shape (389, 14)
jul shape (4, 2)
aug shape (20, 2)


So the July and August csvs seem mostly useless for now

---
## CA Water Suppliers Data
---

In [781]:
print("water shape", water.shape)
water.head()

water shape (32425, 25)


,supplier_name,public_water_system_id,reporting_month,county,hydrologic_region,climate_zone,total_population_served,reference_2014_population,water_shortage_contingency_stage_invoked,water_shortage_level_indicator,...,reported_preliminary_commercial_agricultural_water,reported_final_commercial_agricultural_water,reported_preliminary_commercial_industrial_and_institutional_water,reported_final_commercial_industrial_and_institutional_water,calculated_total_potable_water_production_gallons_ag_excluded,calculated_total_potable_water_production_gallons_2013_ag_excluded,calculated_commercial_agricultural_water_gallons,calculated_commercial_agricultural_water_gallons_2013,calculated_r_gpcd,qualification
0,East Bay Municipal Utilities District,CA0110005,2021-04-15,"Alameda,Contra Costa",San Francisco Bay,3,1400000.0,1379000.0,Stage 0,No,...,NaN,NaN,1026.0,NaN,4.964000e+09,5.418000e+09,0.0,0.0,73.2781,1. To provide contextâ€¦the average R-GPCD for...
1,East Bay Municipal Utilities District,CA0110005,2021-03-15,"Alameda,Contra Costa",San Francisco Bay,3,1400000.0,1379000.0,Stage 0,No,...,NaN,NaN,908.0,NaN,4.221000e+09,4.928000e+09,0.0,0.0,59.3274,1. To provide contextâ€¦the average R-GPCD for...
2,East Bay Municipal Utilities District,CA0110005,2021-02-15,"Alameda,Contra Costa",San Francisco Bay,3,1400000.0,1379000.0,Stage 0,No,...,NaN,NaN,801.0,NaN,3.549000e+09,4.016000e+09,0.0,0.0,55.2268,1. To provide contextâ€¦the average R-GPCD for...
3,East Bay Municipal Utilities District,CA0110005,2021-01-15,"Alameda,Contra Costa",San Francisco Bay,3,1400000.0,1379000.0,Stage 0,No,...,NaN,NaN,902.0,NaN,3.919000e+09,4.099000e+09,0.0,0.0,55.9857,1. To provide contextâ€¦the average R-GPCD for...
4,East Bay Municipal Utilities District,CA0110005,2020-12-15,"Alameda,Contra Costa",San Francisco Bay,3,1450000.0,1379000.0,Stage 0,No,...,NaN,NaN,952.0,579.0,4.169000e+09,4.772000e+09,0.0,0.0,55.6485,1. To provide contextâ€¦the average R-GPCD for...


In [782]:
# water.final_percent_residential_use.mean()

In [783]:
water.columns

Index(['supplier_name', 'public_water_system_id', 'reporting_month', 'county',
       'hydrologic_region', 'climate_zone', 'total_population_served',
       'reference_2014_population', 'water_shortage_contingency_stage_invoked',
       'water_shortage_level_indicator', 'water_production_units',
       'reported_preliminary_total_potable_water_production',
       'reported_final_total_potable_water_production',
       'preliminary_percent_residential_use', 'final_percent_residential_use',
       'reported_preliminary_commercial_agricultural_water',
       'reported_final_commercial_agricultural_water',
       'reported_preliminary_commercial_industrial_and_institutional_water',
       'reported_final_commercial_industrial_and_institutional_water',
       'calculated_total_potable_water_production_gallons_ag_excluded',
       'calculated_total_potable_water_production_gallons_2013_ag_excluded',
       'calculated_commercial_agricultural_water_gallons',
       'calculated_commercial_agri

In [784]:
water['reporting_month'] = pd.to_datetime(water['reporting_month'])
print(water['reporting_month'].max())
print(water['reporting_month'].min())

2021-04-15 00:00:00
2014-06-15 00:00:00


In [785]:
water.isnull().sum()

supplier_name                                                             0
public_water_system_id                                                    0
reporting_month                                                           0
county                                                                    0
hydrologic_region                                                         0
climate_zone                                                              0
total_population_served                                                   0
reference_2014_population                                                 1
water_shortage_contingency_stage_invoked                               1529
water_shortage_level_indicator                                        28686
water_production_units                                                    0
reported_preliminary_total_potable_water_production                   30868
reported_final_total_potable_water_production                           423
preliminary_

In [786]:
water.reported_final_commercial_industrial_and_institutional_water.value_counts()

0.00      469
18.00      60
27.00      28
39.00      28
28.00      28
         ... 
580.70      1
240.83      1
119.73      1
29.57       1
111.30      1
Name: reported_final_commercial_industrial_and_institutional_water, Length: 16843, dtype: int64

In [787]:
water.final_percent_residential_use.describe()

count    31981.000000
mean        69.734913
std         15.091413
min          0.050000
25%         61.370000
50%         70.500000
75%         80.000000
max        121.000000
Name: final_percent_residential_use, dtype: float64

In [788]:
water[water.final_percent_residential_use > 100]

,supplier_name,public_water_system_id,reporting_month,county,hydrologic_region,climate_zone,total_population_served,reference_2014_population,water_shortage_contingency_stage_invoked,water_shortage_level_indicator,...,reported_preliminary_commercial_agricultural_water,reported_final_commercial_agricultural_water,reported_preliminary_commercial_industrial_and_institutional_water,reported_final_commercial_industrial_and_institutional_water,calculated_total_potable_water_production_gallons_ag_excluded,calculated_total_potable_water_production_gallons_2013_ag_excluded,calculated_commercial_agricultural_water_gallons,calculated_commercial_agricultural_water_gallons_2013,calculated_r_gpcd,qualification
2136,Lake Hemet Municipal Water District,CA3310022,2019-12-15,Riverside,South Coast,10,52914.0,52914.0,3,NaN,...,NaN,NaN,NaN,42.590,129363017.0,258400183.0,0.0,0.0,79.4947,NaN
7438,Joshua Basin Water District,CA3610025,2019-09-15,San Bernardino,Colorado River,14,9665.0,9514.0,Stage No. 1 (Ordinance 15-9),NaN,...,NaN,0.0,NaN,18.905,39753874.0,51810377.0,0.0,0.0,140.2690,"Total Population Served: On April 11, 2019, JB..."
29001,South Feather Water and Power Agency,"CA0410006,CA0410012",2020-11-15,Butte,Sacramento River,11,16770.0,16346.0,0,No,...,0.0,0.0,NaN,15.370,116000000.0,110000000.0,0.0,0.0,253.6280,"Because of the fires during September, the met..."
29002,South Feather Water and Power Agency,"CA0410006,CA0410012",2020-10-15,Butte,Sacramento River,11,16770.0,16346.0,0,No,...,0.0,0.0,0.0,33.500,205000000.0,164000000.0,0.0,0.0,477.1390,Our customer base was affected by the North Co...
31607,San Bernardino County Service Area 70J,CA3610125,2020-11-15,San Bernardino,South Coast,14,12451.0,11407.0,0,No,...,NaN,0.0,NaN,0.000,37147063.0,35843657.0,0.0,0.0,110.9050,Report is based on available data due to billi...


Only 5 entries where residential use exceeds 100%

In [789]:
len(conserv[' Hydrologic Region'].unique())

10

In [790]:
len(water['hydrologic_region'].unique())

10

In [791]:
true_count = 0
for i in conserv[' Hydrologic Region'].unique():
    if i in water['hydrologic_region'].unique():
        true_count += 1
true_count

10

This is good, they match up in totality 

---
## Water Suppliers Cleaning
---

In [792]:
# dropping columns where over half the 
water = water.T[water.isnull().sum() < water.shape[0]//2]
water = water.T
water.isnull().sum()

supplier_name                                                            0
public_water_system_id                                                   0
reporting_month                                                          0
county                                                                   0
hydrologic_region                                                        0
climate_zone                                                             0
total_population_served                                                  0
reference_2014_population                                                1
water_shortage_contingency_stage_invoked                              1529
water_production_units                                                   0
reported_final_total_potable_water_production                          423
final_percent_residential_use                                          444
reported_final_commercial_industrial_and_institutional_water          7013
calculated_total_potable_

In [793]:
water.supplier_name.nunique()

413

In [794]:
water.public_water_system_id.nunique()

413

So these are the same

In [795]:
water.water_shortage_contingency_stage_invoked.nunique()

712

In [796]:
water.water_shortage_contingency_stage_invoked.value_counts()[:30]

2                                                     5868
1                                                     5813
3                                                     2176
0                                                     2129
Stage 2                                               1336
Stage 1                                               1081
None                                                   834
Stage 2 of the WSCP                                    833
Level 1                                                487
none                                                   408
Stage 2 of the WSCP; Stage 1 of CPUC Schedule 14.1     374
Level 2                                                307
Stage 3                                                275
4                                                      272
Stage II                                               269
II                                                     249
CPUC Rule 14.1 Stage 1                                 2

This looks like there are only 3 or 4 stages just recorded in different ways and could be encoded with some effort

In [797]:
water['public_water_system_id'].map(len).value_counts()

9      28486
19      1874
29       807
39       387
49       241
59       216
69       166
109       83
99        83
79        82
Name: public_water_system_id, dtype: int64

In [798]:
water[water['public_water_system_id'].map(len) > 59]['supplier_name'].value_counts()

Placer County Water Agency                               83
California Water Service Company Kern River Valley       83
California-American Water Company Sacramento District    83
Tuolumne Utilities District                              83
Sacramento County Water Agency                           82
Name: supplier_name, dtype: int64

In [799]:
water[water['public_water_system_id'].map(len) > 100]

,supplier_name,public_water_system_id,reporting_month,county,hydrologic_region,climate_zone,total_population_served,reference_2014_population,water_shortage_contingency_stage_invoked,water_production_units,reported_final_total_potable_water_production,final_percent_residential_use,reported_final_commercial_industrial_and_institutional_water,calculated_total_potable_water_production_gallons_ag_excluded,calculated_total_potable_water_production_gallons_2013_ag_excluded,calculated_commercial_agricultural_water_gallons,calculated_commercial_agricultural_water_gallons_2013,calculated_r_gpcd
7585,Tuolumne Utilities District,"CA5510001,CA5510012,CA5500363,CA5510002,CA5510...",2021-04-15,Tuolumne,San Joaquin River,12,31110,28997,Phase II,AF,NaN,NaN,NaN,1.12745e+08,1.1307e+08,0,0,83.3534
7586,Tuolumne Utilities District,"CA5510001,CA5510012,CA5500363,CA5510002,CA5510...",2021-03-15,Tuolumne,San Joaquin River,12,31110,28997,Phase II,AF,NaN,NaN,NaN,8.66765e+07,9.28677e+07,0,0,53.9251
7587,Tuolumne Utilities District,"CA5510001,CA5510012,CA5500363,CA5510002,CA5510...",2021-02-15,Tuolumne,San Joaquin River,12,31110,28997,Phase II,AF,NaN,NaN,NaN,8.60248e+07,7.98336e+07,0,0,59.2539
7588,Tuolumne Utilities District,"CA5510001,CA5510012,CA5500363,CA5510002,CA5510...",2021-01-15,Tuolumne,San Joaquin River,12,31110,28997,Phase II,AF,NaN,64,NaN,8.53731e+07,9.41711e+07,0,0,56.6551
7589,Tuolumne Utilities District,"CA5510001,CA5510012,CA5500363,CA5510002,CA5510...",2020-12-15,Tuolumne,San Joaquin River,12,31110,28997,Phase II,AF,NaN,NaN,NaN,8.40697e+07,1.06879e+08,0,0,58.4053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,Tuolumne Utilities District,"CA5510001,CA5510012,CA5500363,CA5510002,CA5510...",2014-10-15,Tuolumne,San Joaquin River,12,28997,28997,Phase II of IV,AF,365.5,81.6,NaN,1.19262e+08,1.4924e+08,0,0,108.262
7664,Tuolumne Utilities District,"CA5510001,CA5510012,CA5500363,CA5510002,CA5510...",2014-09-15,Tuolumne,San Joaquin River,12,28997,28997,2 of 4,AF,413,81.6,NaN,1.34577e+08,1.87527e+08,0,0,126.237
7665,Tuolumne Utilities District,"CA5510001,CA5510012,CA5500363,CA5510002,CA5510...",2014-08-15,Tuolumne,San Joaquin River,12,28997,28997,NaN,AF,444,81.6,NaN,1.44678e+08,2.1995e+08,0,0,131.334
7666,Tuolumne Utilities District,"CA5510001,CA5510012,CA5500363,CA5510002,CA5510...",2014-07-15,Tuolumne,San Joaquin River,12,28997,28997,Phase III of IV,AF,440,81.6,NaN,1.43375e+08,2.35428e+08,0,0,130.151


In [800]:
water[water['public_water_system_id'].map(len) > 100][:1]['public_water_system_id'].str.split(',')

7585    [CA5510001, CA5510012, CA5500363, CA5510002, C...
Name: public_water_system_id, dtype: object

In [801]:
a = water[water['public_water_system_id'].map(len) > 100][:1]['public_water_system_id'].values[0]
a = a.split(',')
a

['CA5510001',
 'CA5510012',
 'CA5500363',
 'CA5510002',
 'CA5510003',
 'CA5510013',
 'CA5510015',
 'CA5510021',
 'CA5510025',
 'CA5510028',
 'CA5510033']

Checking to see if there is overlap with any other suppliers

In [802]:
# checking for system id CA5510013
water[water['public_water_system_id'].str.contains('CA5510013')]['supplier_name'].value_counts()

Tuolumne Utilities District    83
Name: supplier_name, dtype: int64

In [803]:
# checking for system id CA5510001
water[water['public_water_system_id'].str.contains('CA5510001')]['supplier_name'].value_counts()

Tuolumne Utilities District    83
Name: supplier_name, dtype: int64

In [804]:
b = water[water['public_water_system_id'].map(len) == 59][:1]['public_water_system_id'].values[0]
b = b.split(',')
b

['CA0510004', 'CA0510005', 'CA0510006', 'CA0510016', 'CA0510017', 'CA0510019']

In [805]:
# checking for system id CA0510004
water[water['public_water_system_id'].str.contains('CA0510004')]['supplier_name'].value_counts()

Calaveras County Water District    50
Name: supplier_name, dtype: int64

In [806]:
len(b[0])
b[0][2:9]

'0510004'

Looks like each one covering multiple system id's is unique and the first value could be used for encoding 

In [807]:
water['system_id'] = water['public_water_system_id'].map(lambda x: int(str(x)[2:9]))
water.system_id

0         110005
1         110005
2         110005
3         110005
4         110005
          ...   
32420    1910149
32421     410008
32422     410008
32423     410008
32424     410008
Name: system_id, Length: 32425, dtype: int64

In [808]:
water.head()

,supplier_name,public_water_system_id,reporting_month,county,hydrologic_region,climate_zone,total_population_served,reference_2014_population,water_shortage_contingency_stage_invoked,water_production_units,reported_final_total_potable_water_production,final_percent_residential_use,reported_final_commercial_industrial_and_institutional_water,calculated_total_potable_water_production_gallons_ag_excluded,calculated_total_potable_water_production_gallons_2013_ag_excluded,calculated_commercial_agricultural_water_gallons,calculated_commercial_agricultural_water_gallons_2013,calculated_r_gpcd,system_id
0,East Bay Municipal Utilities District,CA0110005,2021-04-15,"Alameda,Contra Costa",San Francisco Bay,3,1.4e+06,1.379e+06,Stage 0,MG,NaN,NaN,NaN,4.964e+09,5.418e+09,0,0,73.2781,110005
1,East Bay Municipal Utilities District,CA0110005,2021-03-15,"Alameda,Contra Costa",San Francisco Bay,3,1.4e+06,1.379e+06,Stage 0,MG,NaN,NaN,NaN,4.221e+09,4.928e+09,0,0,59.3274,110005
2,East Bay Municipal Utilities District,CA0110005,2021-02-15,"Alameda,Contra Costa",San Francisco Bay,3,1.4e+06,1.379e+06,Stage 0,MG,NaN,NaN,NaN,3.549e+09,4.016e+09,0,0,55.2268,110005
3,East Bay Municipal Utilities District,CA0110005,2021-01-15,"Alameda,Contra Costa",San Francisco Bay,3,1.4e+06,1.379e+06,Stage 0,MG,NaN,NaN,NaN,3.919e+09,4.099e+09,0,0,55.9857,110005
4,East Bay Municipal Utilities District,CA0110005,2020-12-15,"Alameda,Contra Costa",San Francisco Bay,3,1.45e+06,1.379e+06,Stage 0,MG,4169.2,60,579,4.169e+09,4.772e+09,0,0,55.6485,110005


In [809]:
water.dtypes

supplier_name                                                                 object
public_water_system_id                                                        object
reporting_month                                                       datetime64[ns]
county                                                                        object
hydrologic_region                                                             object
climate_zone                                                                  object
total_population_served                                                       object
reference_2014_population                                                     object
water_shortage_contingency_stage_invoked                                      object
water_production_units                                                        object
reported_final_total_potable_water_production                                 object
final_percent_residential_use                                    

In [810]:
counties = {county: idx for idx, county in enumerate(water['county'].unique())}
hydro_regions = {hydro: idx for idx, hydro in enumerate(water['hydrologic_region'].unique())}
water['county_enc'] = water['county'].map(counties)
water['hydro_region_enc'] = water['hydrologic_region'].map(hydro_regions)

In [811]:
counties = {county: idx for idx, county in enumerate(water['county'].unique())}
hydro_regions = {hydro: idx for idx, hydro in enumerate(water['hydrologic_region'].unique())}
water['county_enc'] = water['county'].map(counties)
water['hydro_region_enc'] = water['hydrologic_region'].map(hydro_regions)

In [812]:
water['total_population_served'].astype(int)

0        1400000
1        1400000
2        1400000
3        1400000
4        1450000
          ...   
32420      11712
32421      10312
32422      10312
32423      10312
32424      10312
Name: total_population_served, Length: 32425, dtype: int64

In [813]:
water.head()

,supplier_name,public_water_system_id,reporting_month,county,hydrologic_region,climate_zone,total_population_served,reference_2014_population,water_shortage_contingency_stage_invoked,water_production_units,...,final_percent_residential_use,reported_final_commercial_industrial_and_institutional_water,calculated_total_potable_water_production_gallons_ag_excluded,calculated_total_potable_water_production_gallons_2013_ag_excluded,calculated_commercial_agricultural_water_gallons,calculated_commercial_agricultural_water_gallons_2013,calculated_r_gpcd,system_id,county_enc,hydro_region_enc
0,East Bay Municipal Utilities District,CA0110005,2021-04-15,"Alameda,Contra Costa",San Francisco Bay,3,1.4e+06,1.379e+06,Stage 0,MG,...,NaN,NaN,4.964e+09,5.418e+09,0,0,73.2781,110005,0,0
1,East Bay Municipal Utilities District,CA0110005,2021-03-15,"Alameda,Contra Costa",San Francisco Bay,3,1.4e+06,1.379e+06,Stage 0,MG,...,NaN,NaN,4.221e+09,4.928e+09,0,0,59.3274,110005,0,0
2,East Bay Municipal Utilities District,CA0110005,2021-02-15,"Alameda,Contra Costa",San Francisco Bay,3,1.4e+06,1.379e+06,Stage 0,MG,...,NaN,NaN,3.549e+09,4.016e+09,0,0,55.2268,110005,0,0
3,East Bay Municipal Utilities District,CA0110005,2021-01-15,"Alameda,Contra Costa",San Francisco Bay,3,1.4e+06,1.379e+06,Stage 0,MG,...,NaN,NaN,3.919e+09,4.099e+09,0,0,55.9857,110005,0,0
4,East Bay Municipal Utilities District,CA0110005,2020-12-15,"Alameda,Contra Costa",San Francisco Bay,3,1.45e+06,1.379e+06,Stage 0,MG,...,60,579,4.169e+09,4.772e+09,0,0,55.6485,110005,0,0


In [814]:
water.reported_final_total_potable_water_production

0            NaN
1            NaN
2            NaN
3            NaN
4         4169.2
          ...   
32420      180.7
32421    77407.5
32422    45997.3
32423    39363.6
32424    44901.7
Name: reported_final_total_potable_water_production, Length: 32425, dtype: object

In [815]:
water.columns

Index(['supplier_name', 'public_water_system_id', 'reporting_month', 'county',
       'hydrologic_region', 'climate_zone', 'total_population_served',
       'reference_2014_population', 'water_shortage_contingency_stage_invoked',
       'water_production_units',
       'reported_final_total_potable_water_production',
       'final_percent_residential_use',
       'reported_final_commercial_industrial_and_institutional_water',
       'calculated_total_potable_water_production_gallons_ag_excluded',
       'calculated_total_potable_water_production_gallons_2013_ag_excluded',
       'calculated_commercial_agricultural_water_gallons',
       'calculated_commercial_agricultural_water_gallons_2013',
       'calculated_r_gpcd', 'system_id', 'county_enc', 'hydro_region_enc'],
      dtype='object')

In [816]:
convert_to_int = ['climate_zone', 'total_population_served',
       'reference_2014_population', 'reported_final_total_potable_water_production',
       'final_percent_residential_use', 'reported_final_commercial_industrial_and_institutional_water',
       'calculated_total_potable_water_production_gallons_ag_excluded',
       'calculated_total_potable_water_production_gallons_2013_ag_excluded',
       'calculated_commercial_agricultural_water_gallons',
       'calculated_commercial_agricultural_water_gallons_2013',
       'calculated_r_gpcd']
water[convert_to_int].dtypes

climate_zone                                                          object
total_population_served                                               object
reference_2014_population                                             object
reported_final_total_potable_water_production                         object
final_percent_residential_use                                         object
reported_final_commercial_industrial_and_institutional_water          object
calculated_total_potable_water_production_gallons_ag_excluded         object
calculated_total_potable_water_production_gallons_2013_ag_excluded    object
calculated_commercial_agricultural_water_gallons                      object
calculated_commercial_agricultural_water_gallons_2013                 object
calculated_r_gpcd                                                     object
dtype: object

In [817]:
water[convert_to_int].isnull().sum()
water['final_percent_residential_use'] - water['calculated_total_potable_water_production_gallons_2013_ag_excluded']

0                NaN
1                NaN
2                NaN
3                NaN
4         -4.772e+09
            ...     
32420            NaN
32421   -5.65999e+07
32422   -3.91406e+07
32423   -2.89034e+07
32424   -3.30182e+07
Length: 32425, dtype: object

In [818]:
for i in convert_to_int:
    # coerce errors as per https://stackoverflow.com/questions/47333227/pandas-valueerror-cannot-convert-float-nan-to-integer
    water[i] = pd.to_numeric(water[i], errors='coerce')
water[convert_to_int].isnull().sum()

climate_zone                                                             0
total_population_served                                                  0
reference_2014_population                                                1
reported_final_total_potable_water_production                          423
final_percent_residential_use                                          444
reported_final_commercial_industrial_and_institutional_water          7013
calculated_total_potable_water_production_gallons_ag_excluded            0
calculated_total_potable_water_production_gallons_2013_ag_excluded       1
calculated_commercial_agricultural_water_gallons                         0
calculated_commercial_agricultural_water_gallons_2013                    5
calculated_r_gpcd                                                        0
dtype: int64

In [819]:
# mean encoding null values 
for i in convert_to_int:
    water[i] = water[i].fillna(water[i].mean())
water[convert_to_int].isnull().sum()

climate_zone                                                          0
total_population_served                                               0
reference_2014_population                                             0
reported_final_total_potable_water_production                         0
final_percent_residential_use                                         0
reported_final_commercial_industrial_and_institutional_water          0
calculated_total_potable_water_production_gallons_ag_excluded         0
calculated_total_potable_water_production_gallons_2013_ag_excluded    0
calculated_commercial_agricultural_water_gallons                      0
calculated_commercial_agricultural_water_gallons_2013                 0
calculated_r_gpcd                                                     0
dtype: int64

In [820]:
water[convert_to_int].describe().T

,count,mean,std,min,25%,50%,75%,max
climate_zone,32425.0,9.059306e+00,3.796250e+00,1.000000e+00,6.000000e+00,9.000000e+00,1.200000e+01,1.600000e+01
total_population_served,32425.0,9.137515e+04,2.431812e+05,1.000000e+02,2.097000e+04,4.418900e+04,9.332200e+04,4.133849e+06
reference_2014_population,32425.0,8.948902e+04,2.371816e+05,1.120000e+02,2.008000e+04,4.376700e+04,9.275600e+04,3.960940e+06
reported_final_total_potable_water_production,32425.0,2.877158e+07,1.989786e+08,4.030000e+00,2.350000e+02,7.340240e+02,2.669800e+03,5.252463e+09
final_percent_residential_use,32425.0,6.973491e+01,1.498773e+01,5.000000e-02,6.160000e+01,7.007000e+01,8.000000e+01,1.210000e+02
reported_final_commercial_industrial_and_institutional_water,32425.0,6.977072e+06,4.889642e+07,0.000000e+00,6.398000e+01,3.100000e+02,6.977072e+06,1.401330e+09
calculated_total_potable_water_production_gallons_ag_excluded,32425.0,3.722133e+08,8.484037e+08,4.000000e+06,8.178871e+07,1.805217e+08,3.906959e+08,1.658128e+10
calculated_total_potable_water_production_gallons_2013_ag_excluded,32425.0,4.438801e+08,9.875968e+08,7.171076e+06,1.002400e+08,2.190000e+08,4.734621e+08,1.795669e+10
calculated_commercial_agricultural_water_gallons,32425.0,4.070411e+06,4.054954e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.071725e+09
calculated_commercial_agricultural_water_gallons_2013,32425.0,5.588803e+06,5.178750e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.023173e+09


In [821]:
water.dtypes

supplier_name                                                                 object
public_water_system_id                                                        object
reporting_month                                                       datetime64[ns]
county                                                                        object
hydrologic_region                                                             object
climate_zone                                                                   int64
total_population_served                                                      float64
reference_2014_population                                                    float64
water_shortage_contingency_stage_invoked                                      object
water_production_units                                                        object
reported_final_total_potable_water_production                                float64
final_percent_residential_use                                    

In [822]:
water.shape

(32425, 21)

---
## Dimensional analyisis 
---
Most of the columns are in gallons but some of them use different units

In [823]:
water.water_production_units.value_counts()
# Interpreting the following using 

AF     19253
MG      7200
G       3750
CCF     2222
Name: water_production_units, dtype: int64

[Water Measurement Units and Conversion Factors](https://extension.okstate.edu/fact-sheets/water-measurement-units-and-conversion-factors.html)

The Data Dictionary only states these are units for reported production quantities 

The four units of measurement are:
- CCF - one hundred cubic feet of water (748.052 gallons)
- AF -  The volume of water that would cover one acre one foot deep (325,851 gallons)
    - Also this could just maybe mean 'Water AF'
- G - Gallons (... 1 gallon?)
- MG - Mega-Gallons (1,000,000 gallons)

In [824]:
# test_category = 'reported_final_commercial_industrial_and_institutional_water'
test_category = 'reported_final_total_potable_water_production'

g_mean = water[water['water_production_units'] == 'G'][test_category].mean()
mg_mean = water[water['water_production_units'] == 'MG'][test_category].mean()
af_mean = water[water['water_production_units'] == 'AF'][test_category].mean()
ccf_mean = water[water['water_production_units'] == 'CCF'][test_category].mean()

In [825]:
# test_category = 'reported_final_commercial_industrial_and_institutional_water'
test_category = 'reported_final_total_potable_water_production'

g_median = water[water['water_production_units'] == 'G'][test_category].median()
mg_median = water[water['water_production_units'] == 'MG'][test_category].median()
af_median = water[water['water_production_units'] == 'AF'][test_category].median()
ccf_median = water[water['water_production_units'] == 'CCF'][test_category].median()

Looking at mean and median values to verify that conversion units are within the expected range

In [826]:
print('Mean Values:')
print('------------')
print('G', g_mean)
print('MG', mg_mean)
print('AF', af_mean)
print('CCF', ccf_mean)

Mean Values:
------------
G 245640644.60923892
MG 288062.14418535976
AF 400291.0260551646
CCF 893470.0089239818


In [827]:
print('Median Values:')
print('------------')
print('G', g_median)
print('MG', mg_median)
print('AF', af_median)
print('CCF', ccf_median)

Median Values:
------------
G 94143536.5
MG 155.936
AF 731.0
CCF 194559.0


In [828]:
g_median / mg_median

603731.8932126

If MG is mega/million-gallons this seems like it's close to on the mark 

In [829]:
g_median / af_median

128787.32763337894

As one acre-foot is 325,851 gallons this seems somewhat reasonable 

In [830]:
g_median / ccf_median

483.8816837052

One CCF/HCF is 748.052 gallons, so this one seems like it's likely correct

In [831]:
conversion_cols = ['reported_final_total_potable_water_production']

for i in water['water_production_units'].unique():
    print(i)
    print(water[water['water_production_units'] == i]['hydrologic_region'].unique())
    print()

MG
['San Francisco Bay' 'San Joaquin River' 'Tulare Lake' 'Sacramento River'
 'South Coast' 'Colorado River' 'North Lahontan' 'North Coast'
 'Central Coast' 'South Lahontan']

G
['Central Coast' 'San Joaquin River' 'San Francisco Bay' 'Colorado River'
 'North Coast' 'South Lahontan' 'Sacramento River' 'North Lahontan'
 'Tulare Lake' 'South Coast']

AF
['South Coast' 'Sacramento River' 'Colorado River' 'San Francisco Bay'
 'South Lahontan' 'North Coast' 'Central Coast' 'Tulare Lake'
 'San Joaquin River']

CCF
['San Francisco Bay' 'Tulare Lake' 'South Coast' 'North Coast'
 'South Lahontan' 'Colorado River' 'Sacramento River' 'San Joaquin River']



Clearly differences in units used aren't on a hydrologic basis 

In [832]:
# for i in water['water_production_units'].unique():
#     print(i)
#     print(water[water['water_production_units'] == i]['supplier_name'].unique())
#     print()

In [833]:
water.dtypes

supplier_name                                                                 object
public_water_system_id                                                        object
reporting_month                                                       datetime64[ns]
county                                                                        object
hydrologic_region                                                             object
climate_zone                                                                   int64
total_population_served                                                      float64
reference_2014_population                                                    float64
water_shortage_contingency_stage_invoked                                      object
water_production_units                                                        object
reported_final_total_potable_water_production                                float64
final_percent_residential_use                                    

In [834]:
conversions = {
    'CCF': 748.052,
    'AF': 325_851, 
    'MG': 1_000_000, 
    }
list(conversions)

['CCF', 'AF', 'MG']

In [835]:
cols = ['reported_final_total_potable_water_production', 
        'reported_final_commercial_industrial_and_institutional_water']
conversions = {
    'CCF': 748.052,
    'AF': 325_851, 
    'MG': 1_000_000, 
    }
for i in conversions:
    wpu = 'water_production_units'
#     water[water[wpu] == i][cols[0]] = water[water[wpu] == i][cols[0]].copy().map(lambda x: x*conversions[i])
#     water[water[wpu] == i][cols[1]] = water[water[wpu] == i][cols[1]].copy().map(lambda x: x*conversions[i])
    water.loc[water[wpu] == i, cols[0]] = water.loc[water[wpu] == i, cols[0]].copy().map(lambda x: x*conversions[i])
    water.loc[water[wpu] == i, cols[1]] = water.loc[water[wpu] == i, cols[1]].copy().map(lambda x: x*conversions[i])

In [836]:
# test_category = 'reported_final_commercial_industrial_and_institutional_water'
test_category = 'reported_final_total_potable_water_production'

g_median = water[water['water_production_units'] == 'G'][test_category].median()
mg_median = water[water['water_production_units'] == 'MG'][test_category].median()
af_median = water[water['water_production_units'] == 'AF'][test_category].median()
ccf_median = water[water['water_production_units'] == 'CCF'][test_category].median()

In [837]:
g_median / mg_median

0.6037318932126

---
## Water Conservation Cleaning
---

In [838]:
conserv.head()

,Supplier Name,Hydrologic Region,State-mandated Conservation Standard - Previous (3/1/2016),State-mandated Conservation Standard - Current (Effective 6/1/2016),Did Supplier Self-certify?,Enforceable Cumulative Percent Savings (as compared to 2013) - June 2015-Aug 2016,June 2016 to Aug 2016 Self-certified?,Applicable Cumulative Savings by Supplier (as compared to 2013),Missed Conservation Standard By,Order Status,Monthly Savings (as compared to 2013) - Aug 2015,Monthly Savings (as compared to 2013) - Aug 2016,Estimated Aug 2016 Monthly R-GPCD,Total Population Served
0,California City City of,South Lahontan,28%,28%,No,13.1%,18.0%,13.1%,14.9%,C,16.8%,20.5%,148.5,"14,120"
1,Yucaipa Valley Water District,South Coast,34%,20%,Yes,18.3%,7.1%,7.1%,12.9%,NaN,26.4%,11.9%,260.0,"45,018"
2,Sanger City of,Tulare Lake,26%,26%,No,17.3%,12.4%,17.3%,8.7%,NaN,38.0%,17.2%,181.1,"25,664"
3,Adelanto City of,South Lahontan,16%,16%,No,9.3%,-10.3%,9.3%,6.7%,C,20.5%,-30.6%,125.5,"29,023"
4,Red Bluff City of,Sacramento River,33%,33%,No,27.8%,19.5%,27.8%,5.2%,NaN,42.4%,30.8%,202.9,"14,076"


In [839]:
conserv[' Order Status'].value_counts()

A    4
C    3
Name:  Order Status, dtype: int64

In [840]:
conserv[' Order Status'].isnull().sum()

382

In [841]:
conserv = conserv.drop(columns=' Order Status')

In [842]:
perc_cols = [' State-mandated Conservation Standard - Previous (3/1/2016)',
             ' State-mandated Conservation Standard - Current (Effective 6/1/2016)',
             ' Enforceable Cumulative Percent Savings (as compared to 2013) - June 2015-Aug 2016',
             ' June 2016 to Aug 2016 Self-certified?',
             ' Applicable Cumulative Savings by Supplier (as compared to 2013)',
             ' Missed Conservation Standard By',
             ' Monthly Savings (as compared to 2013) - Aug 2015',
             ' Monthly Savings (as compared to 2013) - Aug 2016',] 

for i in perc_cols:
    conserv[i] = conserv[i].str.strip('%').astype(float)


In [843]:
conserv.columns

Index(['Supplier Name', ' Hydrologic Region',
       ' State-mandated Conservation Standard - Previous (3/1/2016)',
       ' State-mandated Conservation Standard - Current (Effective 6/1/2016)',
       ' Did Supplier Self-certify?',
       ' Enforceable Cumulative Percent Savings (as compared to 2013) - June 2015-Aug 2016',
       ' June 2016 to Aug 2016 Self-certified?',
       ' Applicable Cumulative Savings by Supplier (as compared to 2013)',
       ' Missed Conservation Standard By',
       ' Monthly Savings (as compared to 2013) - Aug 2015',
       ' Monthly Savings (as compared to 2013) - Aug 2016',
       ' Estimated Aug 2016 Monthly R-GPCD', ' Total Population Served'],
      dtype='object')

In [844]:
conserv.dtypes

Supplier Name                                                                          object
 Hydrologic Region                                                                     object
 State-mandated Conservation Standard - Previous (3/1/2016)                           float64
 State-mandated Conservation Standard - Current (Effective 6/1/2016)                  float64
 Did Supplier Self-certify?                                                            object
 Enforceable Cumulative Percent Savings (as compared to 2013) - June 2015-Aug 2016    float64
 June 2016 to Aug 2016 Self-certified?                                                float64
 Applicable Cumulative Savings by Supplier (as compared to 2013)                      float64
 Missed Conservation Standard By                                                      float64
 Monthly Savings (as compared to 2013) - Aug 2015                                     float64
 Monthly Savings (as compared to 2013) - Aug 2016           

In [845]:
conserv[' Total Population Served'].isnull().sum()

0

In [846]:
conserv[' Total Population Served'] = conserv[' Total Population Served'].str.split(',').map(''.join).astype(int)

In [847]:
conserv[' Did Supplier Self-certify?'].value_counts()

Yes    361
No      28
Name:  Did Supplier Self-certify?, dtype: int64

In [848]:
conserv[' Did Supplier Self-certify?'] = conserv[' Did Supplier Self-certify?'].map({'Yes': 1, 
                                                                                    'No': 0})

In [849]:
conserv = conserv.rename(columns={' Hydrologic Region': 'hydrologic_region'})

In [850]:
conserv['hydrologic_region'] = conserv['hydrologic_region'].map(hydro_regions)

In [851]:
conserv.columns

Index(['Supplier Name', 'hydrologic_region',
       ' State-mandated Conservation Standard - Previous (3/1/2016)',
       ' State-mandated Conservation Standard - Current (Effective 6/1/2016)',
       ' Did Supplier Self-certify?',
       ' Enforceable Cumulative Percent Savings (as compared to 2013) - June 2015-Aug 2016',
       ' June 2016 to Aug 2016 Self-certified?',
       ' Applicable Cumulative Savings by Supplier (as compared to 2013)',
       ' Missed Conservation Standard By',
       ' Monthly Savings (as compared to 2013) - Aug 2015',
       ' Monthly Savings (as compared to 2013) - Aug 2016',
       ' Estimated Aug 2016 Monthly R-GPCD', ' Total Population Served'],
      dtype='object')

In [852]:
rename = {'Supplier Name': 'supplier_name',
          ' Hydrologic Region': 'hydrologic_region', 
          ' State-mandated Conservation Standard - Previous (3/1/2016)': 'prev_conserve_standard',
          ' State-mandated Conservation Standard - Current (Effective 6/1/2016)': 'current_conserve_standard',
          ' Did Supplier Self-certify?': 'self_cert',
          ' Enforceable Cumulative Percent Savings (as compared to 2013) - June 2015-Aug 2016': 'pct_enforcable_savings',
          ' June 2016 to Aug 2016 Self-certified?': 'self_cert_2016',
          ' Applicable Cumulative Savings by Supplier (as compared to 2013)': 'cumulative_savings',
          ' Missed Conservation Standard By': 'missed_standard_by',
          ' Monthly Savings (as compared to 2013) - Aug 2015': 'monthly_savings_aug_2015',
          ' Monthly Savings (as compared to 2013) - Aug 2016': 'monthly_savings_aug_2016',
          ' Estimated Aug 2016 Monthly R-GPCD': 'r_gpcd_2016', 
          ' Total Population Served': 'pop_served'
         }
conserv = conserv.rename(columns=rename)

---
## Create Cleaned Data Tables
---

In [853]:
# keeping counties and hydrologic reasons for inference 
reverse_counties = {b: a for a, b in counties.items()}
reverse_hydro  = {b: a for a, b in hydro_regions.items()}
counties_df = pd.DataFrame.from_dict(reverse_counties, orient='index')
counties_df = counties_df.rename(columns={0: 'County'})
hydro_region_df =  pd.DataFrame.from_dict(reverse_hydro, orient='index')
hydro_region_df = hydro_region_df.rename(columns={0: 'hydrologic_region'})

In [854]:
counties_df

,County
0,"Alameda,Contra Costa"
1,San Luis Obispo
2,Orange
3,Los Angeles
4,Merced
...,...
57,Siskiyou
58,Mendocino
59,Lassen
60,Tehama


In [855]:
# keeping a list of supplier names for later use 
supplier_names = water[['supplier_name', 'system_id']]
supplier_names = supplier_names.drop_duplicates()
supplier_names = supplier_names.reset_index(drop=True)
supplier_names.head()

,supplier_name,system_id
0,East Bay Municipal Utilities District,110005
1,Morro Bay City of,4010011
2,Yorba Linda Water District,3010037
3,Long Beach City of,1910065
4,"Los Banos, City of",2410005


In [856]:
cleaned_water = water.select_dtypes(np.number)
cleaned_water = cleaned_water.merge(water['reporting_month'], how='inner', 
                                    left_index=True, right_index=True)
cleaned_water = cleaned_water.reset_index(drop=True)
cleaned_water = cleaned_water.rename(columns={"reporting_month": "reporting_date",})
cleaned_water.shape

(32425, 15)

In [857]:
cleaned_water.dtypes

climate_zone                                                                   int64
total_population_served                                                      float64
reference_2014_population                                                    float64
reported_final_total_potable_water_production                                float64
final_percent_residential_use                                                float64
reported_final_commercial_industrial_and_institutional_water                 float64
calculated_total_potable_water_production_gallons_ag_excluded                float64
calculated_total_potable_water_production_gallons_2013_ag_excluded           float64
calculated_commercial_agricultural_water_gallons                             float64
calculated_commercial_agricultural_water_gallons_2013                        float64
calculated_r_gpcd                                                            float64
system_id                                                        

---
## Exports 
---

In [858]:
hydro_region_df.to_csv('./cleaned_datasets/hydrologic_regions.csv', index=False)
counties_df.to_csv('./cleaned_datasets/counties.csv', index=False)
supplier_names.to_csv('./cleaned_datasets/supplier_names.csv', index=False)
cleaned_water.to_csv('./cleaned_datasets/water_suppliers.csv', index=False)
conserv.to_csv('./cleaned_datasets/conservation.csv', index=False)